# Baseline

In [1]:
import math
import time
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from collections import defaultdict

## df 节省内存函数

In [2]:
def reduce_mem(df):
    start_time = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    # 返回的值以字节为单位，转换成以 MB 为单位
    start_mem = df.memery_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_max, c_min = df[col].max(), df[col].min()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memery_usage().sum() / 1024**2
    print(f'现在数据占用大小为: {end_mem}')
    print(f'减少了{100 * (start_mem - end_mem) / start_mem}')
    print(f'花费时间: {(time.time() - start_time) / 60}')
    return df

## 读取采样或全量数据

In [3]:
def get_all_click_sample(data_path: str, sample_nums: int=10000) -> pd.DataFrame:
    """
    训练集中采样一部分数据调试。
    
    Args:
        data_path(`str`): 原数据的存储路径
        sample_nums(`int`): 采样数目（这里由于机器的内存限制，可以采样用户做）

    Returns:
        pd.DataFrame: 返回一个包含采样后的点击日志数据的 DataFrame；
                      该数据只包含随机选择的用户的点击记录，并且去除了重复的点击记录。
    """
    all_click = pd.read_csv(data_path + 'train_click_log.csv')
    all_user_ids = all_click.user_id.unique()

    sample_user_ids = np.random.choice(all_user_ids, size=sample_nums, replace=False) 
    all_click = all_click[all_click['user_id'].isin(sample_user_ids)]
    
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

In [ ]:
def get_all_click_df(data_path: str='../data/', offline: bool=True) -> pd.DataFrame:
    """
    读取点击数据，支持线上和线下数据的获取。
    根据 offline 参数的值决定读取训练集或训练集与测试集的合并数据。

    Args:
        data_path (`str`): 数据存储路径，默认为 './data_raw/'。
        offline (`bool`): 是否仅使用训练集数据。若为 True，则只读取训练集；若为 False，则合并训练集和测试集。

    Returns:
        pd.DataFrame: 返回一个包含用户点击日志的 DataFrame，数据中去除了重复的点击记录。
    """
    all_click = pd.read_csv(data_path + 'train_click_log.csv')
    if not offline:
        tst_click = pd.read_csv(data_path + 'testA_click_log.csv')
        all_click = pd.concat([all_click, tst_click], ignore_index=True)
    
    all_click = all_click.drop_duplicates(['user_id', 'click_article_id', 'click_timestamp'])
    return all_click

In [5]:
# 获取全量训练集
all_click_df = get_all_click_df(offline=False)
all_click_df

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,199999,160417,1507029570190,4,1,17,1,13,1
1,199999,5408,1507029571478,4,1,17,1,13,1
2,199999,50823,1507029601478,4,1,17,1,13,1
3,199998,157770,1507029532200,4,1,17,1,25,5
4,199998,96613,1507029671831,4,1,17,1,25,5
...,...,...,...,...,...,...,...,...,...
1630628,221924,70758,1508211323220,4,3,2,1,25,2
1630629,207823,331116,1508211542618,4,3,2,1,25,1
1630630,207823,234481,1508211850103,4,3,2,1,25,1
1630631,207823,211442,1508212189949,4,3,2,1,25,1


## 获取【用户-文章-点击时间】字典

In [6]:
def get_user_item_time(click_df: pd.DataFrame) -> dict:
    """
    根据点击时间获取用户的点击文章序列。
    函数将每个用户的点击文章和对应的点击时间组合成字典，格式为 
    {user1: {item1: time1, item2: time2, ...}, ...}。

    Args:
        click_df (`pd.DataFrame`): 包含用户点击数据的 DataFrame

    Returns:
        dict: 返回一个字典，其中键是用户 ID，值是另一个字典，包含用户点击的文章及其对应的点击时间。
    """
    click_df = click_df.sort_values('click_timestamp')
    
    def make_item_time_pair(df):
        return list(zip(df['click_article_id'], df['click_timestamp']))
    
    user_item_time_df = click_df.groupby('user_id')[['click_article_id', 'click_timestamp']].apply(
        lambda x: make_item_time_pair(x)
    ).reset_index().rename(columns={0: 'item_time_list'})
    
    # 将用户 ID 和对应的点击文章时间列表转换为字典
    user_item_time_dict = dict(zip(user_item_time_df['user_id'], user_item_time_df['item_time_list']))
    
    return user_item_time_dict

## 获取点击最多的 TopK 个文章

In [7]:
def get_item_topk_click(click_df: pd.DataFrame, k: int) -> pd.Series:
    """
    获取点击次数最多的前 k 个文章 ID。

    Args:
        click_df (`pd.DataFrame`): 包含点击数据的 DataFrame，必须包含 'click_article_id' 列。
        k (`int`): 需要返回的前 k 个文章的数量。

    Returns:
        `pd.Series`: 返回一个包含前 k 个点击次数最多的文章 ID 的 Series。
    """
    return click_df['click_article_id'].value_counts().index[:k]

## ItemCF 的物品相似度计算

In [ ]:
def itemcf_sim(df: pd.DataFrame) -> None:
    """
    计算物品间的相似度并保存相似度矩阵。

    Args:
        df (`pd.DataFrame`): 包含用户点击数据的 DataFrame，必须包含用户和物品的相关信息。

    Returns:
        None: 该函数不返回值，而是将相似度矩阵保存到本地文件。
    """
    # 获取用户-物品-时间字典，格式为 {用户: [(物品, 点击时间), ...]}
    user_item_time_dict = get_user_item_time(df)
    
    # 初始化物品相似度字典和物品计数器
    i2i_sim = {}
    item_cnt = defaultdict(int)
    
    # 遍历用户及其点击的物品和时间
    for user, item_time_list in tqdm(user_item_time_dict.items()):
        for i, i_click_time in item_time_list:
            item_cnt[i] += 1  # 物品出现次数 + 1
            i2i_sim.setdefault(i, {})  # 确保物品 i 的相似度字典存在
            for j, j_click_time in item_time_list:
                if i == j:
                    continue  # 跳过同一物品
                i2i_sim[i].setdefault(j, 0)  # 确保物品 j 的相似度存在
                # 更新物品 i 和 j 的相似度
                i2i_sim[i][j] += 1 / math.log(len(item_time_list) + 1)
    
    # 复制相似度字典以进行归一化处理
    i2i_sim_ = i2i_sim.copy()
    for i, related_items in i2i_sim.items():
        for j, wij in related_items.items():
            # 归一化相似度值
            i2i_sim_[i][j] = wij / math.sqrt(item_cnt[i] * item_cnt[j])

    # 将得到的相似性矩阵保存到本地文件
    pickle.dump(i2i_sim_, open('../tmp_results/itemcf_i2i_sim.pkl', 'wb'))

In [23]:
i2i_sim = itemcf_sim(all_click_df)

100%|██████████| 250000/250000 [00:20<00:00, 12090.34it/s]


## ItemCF 的文章推荐

In [ ]:
def item_based_recommend(user_id: str, 
                          user_item_time_dict: dict, 
                          i2i_sim: dict, 
                          sim_item_topk: int, 
                          recall_item_num: int, 
                          item_topk_click: list) -> dict:
    """
    基于文章协同过滤的召回。

    Args:
        user_id (`str`): 用户 ID。
        user_item_time_dict (`dict`): 字典，记录用户的点击文章序列，格式为 {user1: {item1: time1, item2: time2, ...} ...}。
        i2i_sim (`dict`): 字典，文章相似性矩阵。
        sim_item_topk (`int`): 整数，选择与当前文章最相似的前 k 篇文章。
        recall_item_num (`int`): 整数，最后召回的文章数量。
        item_topk_click (`list`): 列表，点击次数最多的文章列表，用于用户召回补全。

    Returns:
        `dict`: 召回的文章列表，格式为 {item1: score1, item2: score2, ...}。

    注意:
        基于物品的协同过滤（详细请参考上一期推荐系统基础的组队学习），
        在多路召回部分会加上关联规则的召回策略。
    """
    
    # 获取用户历史交互的文章
    user_hist_items = user_item_time_dict[user_id]
    
    item_rank = {}
    for loc, (i, click_time) in enumerate(user_hist_items.items()):
        # 对于用户历史文章 i，获取与之相似的文章 j
        for j, wij in sorted(i2i_sim[i].items(), key=lambda x: x[1], reverse=True)[:sim_item_topk]:
            if j in user_hist_items:
                continue  # 跳过用户已点击的文章
            
            # 更新文章 j 的得分
            item_rank[j] = item_rank.get(j, 0) + wij
    
    # 如果不足 recall_item_num 个，用热门商品补全
    if len(item_rank) < recall_item_num:
        for i, item in enumerate(item_topk_click):
            if item in item_rank:  # 填充的 item 应该不在原来的列表中
                continue
            item_rank[item] = -i - 100  # 随便给个负数就行
            if len(item_rank) == recall_item_num:
                break
    
    # 根据得分排序并返回前 recall_item_num 个文章
    item_rank = sorted(item_rank.items(), key=lambda x: x[1], reverse=True)[:recall_item_num]
    return item_rank

## 给每个用户根据物品的 ItemCF 推荐文章

In [32]:
user_recall_items_dict = defaultdict(dict)

# 获取【用户-文章-点击时间】的字典
user_item_time_dict = get_user_item_time(all_click_df)

# 获取文章相似度
i2i_sim = pickle.load(open('../tmp_results/itemcf_i2i_sim.pkl', 'rb'))

# 相似文章的数量
sim_item_topk = 10

# 召回文章的数量
recall_item_num = 10

# 得到点击次数最多的物品
item_topk_click = get_item_topk_click(all_click_df, k=50)

for user in tqdm(all_click_df['user_id'].unique()):
    user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, i2i_sim, sim_item_topk, recall_item_num, item_topk_click)

100%|██████████| 250000/250000 [55:49<00:00, 74.65it/s]  


## 召回字典转换成 DataFrame

In [35]:
# 将字典的形式转换成 df
user_item_score_list = []

for user, items in tqdm(user_recall_items_dict.items()):
    for item, score in items:
        user_item_score_list.append([user, item, score])

recall_df = pd.DataFrame(user_item_score_list, columns=['user_id', 'click_article_id', 'pred_score'])
recall_df

100%|██████████| 250000/250000 [00:04<00:00, 54033.54it/s]


,user_id,click_article_id,pred_score
0,199999,276970,0.172377
1,199999,158536,0.106969
2,199999,168623,0.103572
3,199999,123909,0.103572
4,199999,286321,0.097774
...,...,...,...
2499995,200000,187005,0.071191
2499996,200000,50573,0.071180
2499997,200000,63344,0.071180
2499998,200000,255153,0.068034


In [13]:
# 生成提交文件
def submit(recall_df: pd.DataFrame, topk: int = 5, model_name: str = None) -> None:
    """
    生成提交文件，将推荐结果格式化并保存为 CSV 文件。

    Args:
        recall_df (`pd.DataFrame`): 包含用户推荐结果的 DataFrame，必须包含 'user_id' 和 'pred_score' 列。
        topk (`int`): 每个用户需要提交的推荐文章数量，默认为 5。
        model_name (`str`): 模型名称，用于生成文件名。

    Returns:
        None: 该函数不返回值，而是将结果保存为 CSV 文件。
    """
    
    # 根据用户 ID 和预测分数排序
    recall_df = recall_df.sort_values(by=['user_id', 'pred_score'])
    
    # 为每个用户的预测分数排名
    recall_df['rank'] = recall_df.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
    
    # 检查每个用户是否都有至少 topk 篇文章
    tmp = recall_df.groupby('user_id').apply(lambda x: x['rank'].max())
    assert tmp.min() >= topk  # 确保每个用户的最大排名大于等于 topk
    
    # 删除预测分数列
    del recall_df['pred_score']
    
    # 选择排名前 topk 的文章，并重塑 DataFrame 结构
    submit = recall_df[recall_df['rank'] <= topk].set_index(['user_id', 'rank']).unstack(-1).reset_index()
    submit.columns = [int(col) if isinstance(col, int) else col for col in submit.columns.droplevel(0)]
    
    # 按照提交格式定义列名
    submit = submit.rename(columns={'': 'user_id',
                                     '1': 'article_1',
                                     '2': 'article_2',
                                     '3': 'article_3',
                                     '4': 'article_4',
                                     '5': 'article_5'})
    
    # 生成保存文件的名称
    save_name = model_name + '_' + datetime.today().strftime('%m-%d') + '.csv'
    
    # 保存结果为 CSV 文件
    submit.to_csv(save_name, index=False, header=True)

In [47]:
# 获取测试集
tst_click = pd.read_csv('../data/testA_click_log.csv')
tst_users = tst_click['user_id'].unique()

# 从所有召回数据中将测试集中的用户选出来
tst_recall = recall_df[recall_df['user_id'].isin(tst_users)]

# 生成提交文件
submit(tst_recall, topk=5, model_name='itemcf_baseline')